This workbook is on an implementation of RSA encryption and decryption.<br>
I learnt the mathematics from <br>
An introduction to pure mathematics by Martin Liebeck <br>
(Chapter 10 to 15)

In [1]:
# Euclidean algorithm
# input int1, int2
# output s, t, d where s*int1 + t*int2 = d and d is hcf(int1, int2)
# chapter 10, theorem 10.1, proposition 10.3

def euclid(int1, int2):
    sign1 = 1 if int1 > 0 else 0 if int1 == 0 else -1
    sign2 = 1 if int2 > 0 else 0 if int2 == 0 else -1
    
    if sign1 == 0 and sign2 == 0:
        return [0, 0, 0]
    elif sign1 == 0:
        return [0, sign2, abs(int2)]
    elif sign2 == 0:
        return [sign1, 0, abs(int1)]
    
    rs_record = [[0, 1], [1, 0]]
    d = None
    modify_index = 1
    dividend, divisor = abs(int1), abs(int2)
    
    while True:
        q = dividend//divisor
        d = dividend%divisor
        rs_record[modify_index][0] -= q*rs_record[modify_index-1][0]
        rs_record[modify_index][1] -= q*rs_record[modify_index-1][1]
        modify_index = (modify_index+1)&1
        if d == 0:
            break
        dividend, divisor = divisor, d
    rs_record[modify_index][0] *= sign1
    rs_record[modify_index][1] *= sign2
    return [*rs_record[modify_index], divisor]

In [2]:
# Successively squares for calculating n^a mod m
# useful when n^a is very large
# return r, 0 <= r < m, n^a = r mod m
# chapter 13, example 13.3

def high_pow_mod(n, a, m):
    r = 1
    n_work = n
    
    while a > 0:
        n_work %= m
        if n_work == 0:
            return 0
        
        use_power = a&1
        if use_power:
            r *= n_work
            r %= m
        n_work **= 2
        a >>= 1
    return r

In [3]:
# Miller's test for a, using a positive base b
# A test for checking whether an integer is a prime number
# A prime always pass the test, a non prime may or may not pass the test depending on the base
# input: a, the possible prime and b, the base for the test 
# return True if a passes Miller's test
# chapter 14, proposition 14.4, example 14.2 (last section of chapter 14)

def millers_test(a, b):
    assert(b % a > 0)
    
    if a == 1:
        return False
    
    #first step Fermat's test
    if high_pow_mod(b, a - 1, a) != 1:
        return False
    
    
    power = a-1
    
    while True:
        if power%2 == 1:
            return True
        else:
            power //= 2
        
        test = high_pow_mod(b, power, a)
        if test + 1 == a:
            return True
        elif test != 1:
            return False
        else:
            pass

In [4]:
#RSA encoding and decoding

#RSA encoding
#chapter 15 - RSA Codes: Encoding

#p and q are primes, they are private

#for encoding, p and q are not given
#only N = p*q and e are public
#e is coprime to (p-1)(q-1)

#The message is a number x < N, and the encoded message is x^e (mod N)
#The input is a string of numbers
#rsa_encode returns a list of numbers (they are the encoded messages)

def rsa_encode(number_string, N, e):
    import math
    segment_length = int(math.log(N,10))
    extra = -len(number_string)%segment_length
    number_string += "0" * extra
    
    encoded_message = []

    for i in range(0,len(number_string),segment_length):
        #print(number_string[i:i+segment_length])
        encoded_message.append(high_pow_mod(int(number_string[i:i+segment_length]), e, N))

    return encoded_message

#for decoding, p and q are given
#The encoded message is s < N, and the decoded message is s^d (mod N)
#d is a positive integer such that de = 1 (mod (p-1)(q-1))
#d can be found with with p, q and Euclidean algorithm

#The input is a list of numbers (the encoded messages)
#rsa_decode returns a string of numbers

def rsa_decode(encoded_message, N, d):
    import math
    segment_length = int(math.log(N,10))
    
    decoded_number_string = "".join(("0"*segment_length+str(high_pow_mod(encoded_number, d, N)))[-segment_length:] for encoded_number in encoded_message)

    return decoded_number_string

#number string text string conversion

def text_to_num(message):
    letter_number_extend = {chr(i):f"{i-31:0>2d}" for i in range(32,127)}
    message_number = "".join(letter_number_extend[ch] for ch in message if ch in letter_number_extend)
    return message_number

def num_to_text(message_number):
    letter_number_extend = {chr(i):f"{i-31:0>2d}" for i in range(32,127)}
    number_letter_extend = {value:key for key, value in letter_number_extend.items()}    
    message = "".join(number_letter_extend[number_substr] if number_substr in number_letter_extend else "" for number_substr in [message_number[i:i+2] for i in range(0,len(message_number),2)])
    return message

In [10]:
#An example

#Step 1 - find private key numbers p, q and the encoding power e


# try to find two probable primes, p ~ 22435*10**50 and q ~ 56123*10**50  
# use Miller's test 500 times, a list of 500 numbers in [2, 10000]

import random

test_bases = random.sample(range(2,10001),500)
#print(test_bases)

p = 22435*10**50+random.randrange(0,10**50,2)+1
#print(p)
while True:
    if all(millers_test(p, base) for base in test_bases):
        break
    else:
        p += 2
        
q = 56123*10**50+random.randrange(0,10**50,2)+1
#print(q)
while True:
    if all(millers_test(q, base) for base in test_bases):
        break
    else:
        q += 2

#also find a new e ~ 10**10

e = 10**10 + 1
while True:
    if all(millers_test(e, base) for base in test_bases):
        break
    else:
        e += 2
        
assert(euclid(e,(p-1)*(q-1))[-1] == 1)

In [12]:
# what are the public keys
N = p*q
print("Public keys:", f"\nN={N}  \ne={e}")

Public keys: 
N=12591468912359059498248259764543445998332161399559087362671403917752296276639408292094486617507767246453913779  
e=10000000019


In [16]:
#Step 2 - encode our message

message = 'The RSA (Rivest-Shamir-Adleman) cryptosystem is a public-key cryptosystem, one of the oldest widely used for secure data transmission. The initialism "RSA" comes from the surnames of Ron Rivest, Adi Shamir and Leonard Adleman, who publicly described the algorithm in 1977. An equivalent system was developed secretly in 1973 at Government Communications Headquarters (GCHQ), the British signals intelligence agency, by the English mathematician Clifford Cocks. That system was declassified in 1997.'

message_number = text_to_num(message)
encoded_message_number = rsa_encode(message_number, N, e)

print("len of message:",len(message))
print("len of converted number string:",len(message_number))

print("Encoded list of numbers:",encoded_message_number,sep="\n")

len of message: 498
len of converted number string: 996
Encoded list of numbers:
[499916591448234400062846951350689755835188438615495734304355931412300292444599079878190693649903438479424925, 8609272760683628256244918801697763510746899352168896437089518261060330957648076866762556397946301349794465491, 11242196450549641128026751079618734625739631459308270779120854864231898414304393227733587733374813440839401488, 11033990708448573398495881970693925651294424050031951298230832842053264288599545293721163407378784329271956947, 2839925520647377231110768232302507935863900272239252326244742657905291450220830165521509850458911301887927145, 3793722032761078260955795265963449683569947012926477384878731165313288258129966870849641748234817803669421512, 6536340799459862407669811955205869631881371184027955194072092352564392110757772326542298758963693492935839907, 10887266849364973057917301433905146941255463617532069293207357407323532094061316206602858757359328860014121761, 1255064362277676848757352063

In [17]:
#Step 3 - decode the encoded message

#d is a positive integer such that de = 1 (mod (p-1)(q-1))

d, t, s = euclid(e, (p-1)*(q-1))
if d < 0:
    d += (-d//((p-1)*(q-1))+1) * (p-1)*(q-1)
    
print("Private keys:", f"\np={p}\nq={q}")
print()
print("Decoding power:", f"\nd={d}")
print()

decoded_message_number = rsa_decode(encoded_message_number, N, d)
decoded_message = num_to_text(decoded_message_number)

print("Decoded message:", decoded_message)

Private keys: 
p=2243546566139104441511031058643492184834247335993299639
q=5612305580101056331245793780438149228649543797383372261

Decoding power: 
d=6411139574194825386376908898225910037059198716692169060110593169784551902754991269956667953493819145798126659

Decoded message: The RSA (Rivest-Shamir-Adleman) cryptosystem is a public-key cryptosystem, one of the oldest widely used for secure data transmission. The initialism "RSA" comes from the surnames of Ron Rivest, Adi Shamir and Leonard Adleman, who publicly described the algorithm in 1977. An equivalent system was developed secretly in 1973 at Government Communications Headquarters (GCHQ), the British signals intelligence agency, by the English mathematician Clifford Cocks. That system was declassified in 1997.


In [15]:
#Step 4 - check the number strings are the same after encoding and decoding
#message_number is without any RSA encoding
#decoded_message_number is the recovered by reversing the RSA encoding applied to message_number

assert(message_number==decoded_message_number[:len(message_number)])
print("Raw number string:", "\n"+message_number)
print()
print("Decoded raw number string:", "\n"+decoded_message_number[:len(message_number)])

#decoded_message_number may be longer than the raw message_number,
#because 0s are added during encoding to avoid encoding 

Raw number string: 
53737001515234010951748770848514527366787483143469777078667910016883908185808490848570780174840166018186677774681476709001688390818580849084857078130180797001807101857370018077697084850188746970779001868470690171808301847068868370016966856601858366798478748484748079150153737001747974857466777484780103515234030168807870840171838078018573700184868379667870840180710151807901517487708485130134697401527366787483016679690145708079668369013469777078667913018873800181866777746877900169708468837467706901857370016677728083748573780174790118262424150134790170828674876677707985018490848570780188668401697087707780817069018470688370857790017479011826242001668501408087708379787079850136807878867974686685748079840141706669828666838570838401094036415010130185737001358374857484730184747279667784017479857077777472707968700166727079689013016790018573700138797277748473017866857370786685746874667901367774717180836901368068768415015373668501849084857078018866840169706877668484747174706901